# Can you find a better way to segment your customers?

## 📖 Motivation (Kyra) 
You work for a medical device manufacturer in Switzerland. Your company manufactures orthopedic devices and sells them worldwide. The company sells directly to individual doctors who use them on rehabilitation and physical therapy patients.

Historically, the sales and customer support departments have grouped doctors by geography. However, the region is not a good predictor of the number of purchases a doctor will make or their support needs.

Your team wants to use a data-centric approach to segmenting doctors to improve marketing, customer service, and product planning. 

## Related Work

The goal of our project is to perform cluster analysis. Since our project uses K-means algorithm as our primary clustering technique, one non-negligible issue with this algorithm is the curse of dimensionality. In other words, when we run K-means algorithm with a large amount of features, then the algorithm is not likely to perform well due to the exponentially increasing vector space. 

One solution to combat the curse of dimensionality is to apply dimensionality reduction on our feature space. Research by Sidharth Mishra published in 2017 proposes the method "Principal Component Analysis" (1). In a nutshell, the method finds the orthogonal components which explains the most variance of the projected data. The dimensionality reduction kicks in when we select the first few components that explain the most variance, and discard the last few components that don't explain much variance.

While PCA is a great method, it is a linear dimensionality reduction, which limits its capacity to transform the data that are not linearly separable. Thus we take a step further to apply nonlinear dimensionality reduction technique. The technique we pick is called "Radial Basis Function PCA", in which a kernel function is applied to our data to make our data separable. The idea behind the kernel function is to project our data to higher dimension where it becomes separable. This technique of applying RBF kernel on PCA has shown to increase accuracy of Self Organizing Map (2).

<br>

Citation: <br>
(1) Mishra, Sidharth & Sarkar, Uttam & Taraphder, Subhash & Datta, Sanjoy & Swain, Devi & Saikhom, Reshma & Panda, Sasmita & Laishram, Menalsh. (2017). Principal Component Analysis. International Journal of Livestock Research. 1. 10.5455/ijlr.20170415115235. 
<br>
(2) Roy, Parthajit and Swati Adhikari. “Radial Basis Function based Self-Organizing Map Model for Clustering Spatial Data using PCA.” (2018).

## Methods
* ...
* Impute the data:
    * For satisfaction, since there is only about 30% of the data missing, we use KNN imputer to keep the distribution similar
    * For complaints and orders, we assume that if the data is not provided, then the doctor has not filed a complaint or made an order from us
* Cluster the data:
    * First use RBF PCA to prevent curse of dimensionality mentioned in related work
    * Use elbow methods to figure out optimal number of pc components and number of clusters
* Explain feature importance:
    * Extract pc1 of each clusters and looks at each feature's explained variance ratio
    * Perform OLS to see which feature most responsible for purchases

## Objectives


### Region ...

### Purchase and Complaints relationship

## New Customer Segmentation

Use google doc table to answer: Why should we cherish them?, In what areas do they need more support from us? How can we make them happier?

What's their general characteristic?

What's their general characteristic

<img src="images/feature_importance.PNG" width="700">

# Appendix

## Data Cleaning & Wrangling

### Setup

In [3]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from pprint import pprint

import statsmodels.api as sm

from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import KernelPCA, PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

### Import Data

In [4]:
#Read in all four at once
doctors = pd.read_csv('data/doctors.csv')
orders = pd.read_csv('data/orders.csv')
complaints = pd.read_csv('data/complaints.csv')
instructions = pd.read_csv('data/instructions.csv')

In [5]:
def clean_satisfaction(sat):
    if sat == '--':
        sat = np.nan
    else:
        sat = float(sat)
    return sat

doctors['Satisfaction'] = doctors['Satisfaction'].apply(clean_satisfaction)

def transform_rank(rank):
    ###Takes name of doctor's rank and transforms it into ordinal data from 1-9
    if rank == 'Ambassador':
        num_rank = 9
    elif rank == 'Titanium Plus':
        num_rank = 8
    elif rank == 'Titanium':
        num_rank = 7
    elif rank == 'Platinum Plus':
        num_rank = 6
    elif rank == 'Platinum':
        num_rank = 5
    elif rank == 'Gold Plus':
        num_rank = 4
    elif rank == 'Gold':
        num_rank = 3
    elif rank == 'Silver Plus':
        num_rank = 2
    elif rank == 'Silver':
        num_rank = 1
    else:
        num_rank = np.nan
    return num_rank

def conv_cat_to_num(cat):
    ###Takes category of doctor and returns 1 if specialist and 0 if GP
    if cat == 'Specialist':
        cat = 1
    elif cat == 'General Practitioner':
        cat = 0
    else:
        cat = np.nan
    return cat

#apply to doctors dataframe
doctors['Rank'] = doctors['Rank'].apply(transform_rank)
doctors['Category'] = doctors['Category'].apply(conv_cat_to_num)

In [6]:
ords_per_doc = orders['DoctorID'].value_counts()
ords_per_doc = pd.DataFrame(ords_per_doc)
ords_per_doc.index.name = 'DoctorID'
ords_per_doc.columns = ['Orders']
ords_per_doc.reset_index(inplace=True)


In [7]:
doc_IDs = complaints['DoctorID'].unique()
doc_IDs = list(doc_IDs)
comp_per_doc = pd.DataFrame(doc_IDs)
comp_per_doc.columns = ['DoctorID']
comp_per_doc['Total Complaints'] = 0

for ID in doc_IDs:
    temp_df = complaints[complaints['DoctorID'] == ID]
    total_comp = temp_df['Qty'].sum()
    index = comp_per_doc.index[comp_per_doc['DoctorID'] == ID].tolist()[0]
    comp_per_doc.iloc[index, 1] = total_comp



In [8]:
def instr_conv_to_number(str_in):
    if str_in == 'Yes':
        result = 1
    elif str_in == 'No':
        result = 0
    else:
        result = np.nan
    return result

instructions['Instructions'] = instructions['Instructions'].apply(instr_conv_to_number)

In [9]:
doc_merged = doctors.merge(comp_per_doc, how = 'left', on = 'DoctorID')
doc_merged = doc_merged.merge(ords_per_doc, how = 'left', on = 'DoctorID')
doc_merged = doc_merged.merge(instructions, how = 'left', on = 'DoctorID')

doc_merged = doc_merged [['DoctorID',
                          'Satisfaction', 
                          'Category', 
                          'Incidence rate', 
                          'R rate', 
                          'Experience', 
                          'Purchases', 
                          'Total Complaints', 
                          'Orders', 
                          'Instructions']]

In [10]:
doc_merged.head()

,DoctorID,Satisfaction,Category,Incidence rate,R rate,Experience,Purchases,Total Complaints,Orders,Instructions
0,AHDCBA,53.85,1,49.0,0.90,1.20,49.0,NaN,NaN,1.0
1,ABHAHF,100.00,0,37.0,0.00,0.00,38.0,NaN,NaN,NaN
2,FDHFJ,NaN,1,33.0,1.53,0.00,34.0,NaN,NaN,NaN
3,BJJHCA,NaN,1,28.0,2.03,0.48,29.0,NaN,NaN,NaN
4,FJBEA,76.79,1,23.0,0.96,0.75,24.0,NaN,NaN,NaN


### Imputation

In [11]:
df = doc_merged.drop(['DoctorID', 'Instructions'], axis=1)
df['Total Complaints'] = df['Total Complaints'].fillna(0)
df['Orders'] = df['Orders'].fillna(0)
df['Satisfaction'] = KNNImputer(n_neighbors=4).fit_transform(np.array(df['Satisfaction'])[:, None])

### Transform


In [12]:
class IdentityTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, input_array, y=None):
        return self
    
    def transform(self, input_array, y=None):
        return input_array*1

scaled_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])
identity_pipeline = Pipeline([
        ('identity', IdentityTransformer()),
    ])
scaler = ColumnTransformer([
        ('scaled', scaled_pipeline, ["Satisfaction", "Incidence rate", "R rate", "Purchases", "Total Complaints", "Orders"]),
        ('identity', identity_pipeline, ["Experience", "Category"]),
    ])  # transform columnwise and feature is on 2nd dimension


## Clustering

In [13]:

pca = KernelPCA(n_components=3, random_state=22, kernel='rbf')
km = KMeans(n_clusters=3, init="k-means++", n_init=50, max_iter=500, random_state=22)
clusterer = Pipeline([
           ('scaler', scaler),
           ('pca', pca),
           ('kmeans', km)])
_ = clusterer.fit(df)

In [14]:
cluster_df = df.copy().assign(Cluster=clusterer['kmeans'].labels_)
cluster_df.head()

,Satisfaction,Category,Incidence rate,R rate,Experience,Purchases,Total Complaints,Orders,Cluster
0,53.85000,1,49.0,0.90,1.20,49.0,0.0,0.0,0
1,100.00000,0,37.0,0.00,0.00,38.0,0.0,0.0,0
2,29.21872,1,33.0,1.53,0.00,34.0,0.0,0.0,0
3,29.21872,1,28.0,2.03,0.48,29.0,0.0,0.0,0
4,76.79000,1,23.0,0.96,0.75,24.0,0.0,0.0,0
